In [ ]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats as stats
import datetime as dt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

/home/dlhogan/.conda/envs/soil_moisture_env/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.11.1-CAPI-1.17.1). Conversions between both will be slow.
  warnings.warn(


### Import data 

In [3]:
# import UCRB boundaries
#EPSG code
co_epsg = 32613

# pull in UCRB basin 
ucrb_basin_boundary = gpd.read_file('../basin_characteristics/Upper_Colorado_River_Basin_Boundary.json')
ucrb_basin_boundary = ucrb_basin_boundary.to_crs(co_epsg)

In [22]:
# Daily and annual HCDN streamflow and basin characteristics
hcdn_daily_q = xr.open_dataset('../data/streamflowdata/hcdn_basin_chars_and_daily_q.nc')
hcdn_annual_q = xr.open_dataset('../data/streamflowdata/hcdn_basin_annual_q.nc')

# HCDN seasonal precipitation
hcdn_seasonal_ppt = xr.open_dataset('../data/precipdata/prism_hcdn_seasonal_ppt.nc')
hcdn_seasonal_ppt = hcdn_seasonal_ppt.sel(dim_1='ppt')
hcdn_seasonal_ppt = hcdn_seasonal_ppt.rename({'__xarray_dataarray_variable__':'ppt','variable':'site_no'})
hcdn_seasonal_ppt = hcdn_seasonal_ppt.where(~hcdn_seasonal_ppt.site_no.isin(['09306242','09378170','09378630']),drop=True)
hcdn_seasonal_ppt['year'] = hcdn_seasonal_ppt.month.dt.year.where(hcdn_seasonal_ppt.month.dt.month<10, hcdn_seasonal_ppt.month.dt.year+1)
hcdn_annual_ppt = hcdn_seasonal_ppt.groupby('year').sum()

# UCRB monthly precipitation
prism_4km_all_ds = xr.open_dataset('../data/precipdata/prism_4km_ucrb.nc')
prism_4km_all_ds = prism_4km_all_ds.rename({'__xarray_dataarray_variable__':'ppt'})
p_mean = prism_4km_all_ds.where(prism_4km_all_ds>0).mean(dim=['x','y'])
p_mean['WY'] = p_mean.month.dt.year.where(p_mean.month.dt.month<10, p_mean.month.dt.year+1)
annual_p_sum = p_mean.groupby('WY').sum()
annual_p_km3 = (annual_p_sum / 1000) * np.full(len(annual_p_sum.WY), ucrb_basin_boundary.area) / 1e9
annual_p_km3 = annual_p_km3.sel(WY=slice(1896,2022))
annual_p_km3 = annual_p_km3.rename({'ppt':'p_km3'})

# UCRB naturalized streamflow
ucrb_naturalized_streamflow = pd.read_csv('../data/streamflowdata/LF_NATURALIZED_FLOW.csv', header=None, index_col=0)
ucrb_naturalized_streamflow = ucrb_naturalized_streamflow.rename(columns={1:'annual_flow_km3'}) * 1.23348e-6
ucrb_naturalized_streamflow['q_mm_day'] = ucrb_naturalized_streamflow['annual_flow_km3'] * 1e9 / 365 /ucrb_basin_boundary.area.values[0] * 1000
ucrb_naturalized_streamflow['annual_flow_anom'] = (ucrb_naturalized_streamflow['annual_flow_km3']-ucrb_naturalized_streamflow['annual_flow_km3'].mean())/ucrb_naturalized_streamflow['annual_flow_km3'].std()

: 

: 

### Hypothesis testing for change in the flow center of mass between pre- and post-2000 periods

In [15]:
# Zero is the mid elevation basins, 1 is the low elevation basins, and 2 are the high elevation basins
basin_elevation_groups = pd.read_csv('../data/geodata/basin_elevation_groups.csv', index_col=1).drop(columns='Unnamed: 0')
basin_elevation_groups.index =  ['0'+str(site) for site in basin_elevation_groups.index.values]
basin_group_da = basin_elevation_groups.to_xarray().rename({'index':'site_no'})

# Merge this data array onto the datasets for use
hcdn_daily_q = hcdn_daily_q.merge(basin_group_da)
hcdn_annual_q = hcdn_annual_q.merge(basin_group_da)
hcdn_seasonal_ppt = hcdn_seasonal_ppt.merge(basin_group_da)
hcdn_annual_ppt = hcdn_seasonal_ppt.merge(basin_group_da)

<xarray.Dataset>
Dimensions:            (site_no: 26, date: 43098)
Coordinates:
  * site_no            (site_no) object '09034900' '09035900' ... '09361500'
  * date               (date) datetime64[ns] 1901-10-01 ... 2022-09-30
Data variables:
    00060_Mean         (site_no, date) float64 ...
    basin_area         (site_no) float64 ...
    elevation          (site_no) float32 ...
    north              (site_no) float64 ...
    south              (site_no) float64 ...
    aspect             (site_no) object ...
    elevation_range    (site_no) float64 ...
    x                  (site_no) float64 ...
    y                  (site_no) float64 ...
    q_mm_day           (site_no, date) float64 ...
    WY                 (date) int64 ...
    predicted_cluster  (site_no) int64 2 0 0 2 0 2 0 2 0 0 ... 0 1 1 1 1 1 1 2 0

<xarray.Dataset>
Dimensions:  (site_no: 26, month: 511)
Coordinates:
  * month    (month) datetime64[ns] 1894-12-01 1895-03-01 ... 2022-06-01
    dim_1    <U3 'ppt'
  * site_no  (site_no) object '09034900' '09035900' ... '09352900' '09361500'
Data variables:
    ppt      (site_no, month) float64 420.2 256.0 277.2 ... 286.9 115.1 254.5
    year     (month) int64 1895 1895 1895 1895 1896 ... 2021 2021 2022 2022 2022

1) High Elevation Basins

2) Mid Elevation Basins

3) Low Elevation Basins

### Hypothesis testing for change in annual mean flow between pre- and post-2000 periods

1) High Elevation Basins

2) Mid Elevation Basins

3) Low Elevation Basins

4. Upper Colorado River Basin

### Hypothesis testing for change in the distribution of seasonal precipitation in pre- and post-2000 periods

1) High Elevation Basins

2) Mid Elevation Basins

3) Low Elevation Basins

4. Upper Colorado River Basin

### Hypothesis testing for change in the amount of annual precipitation in pre- and post-2000 periods

1) High Elevation Basins

2) Mid Elevation Basins

3) Low Elevation Basins

4. Upper Colorado River Basin

### ANOVA testing for differences between basins in pre- and post-2000 streamflow